In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
import os
import time

from keras.layers import Input, Dense,MaxPooling2D,Flatten, Conv2D,BatchNormalization,Activation
from keras.models import Model

In [13]:
train_data_dir = '/home/asya/Documents/CodePractice/SpeedChallenge/speed_challenge_2017/data/Train/'
dev_data_dir = '/home/asya/Documents/CodePractice/SpeedChallenge/speed_challenge_2017/data/Dev/'
test_data_dir = '/home/asya/Documents/CodePractice/SpeedChallenge/speed_challenge_2017/data/Test/'

# dimensions of the images.
#img_width, img_height = 240, 150

N_train_samples = 47922
N_dev_samples = 4018
N_test_samples = 10798

print(N_train_samples+N_dev_samples)

51940


In [22]:
## Load train images from directory
train_array = []
i_file = 0
for f in range(N_train_samples):
    file = train_data_dir+'Im'+str(i_file)+'.png'
    train_array.append(cv2.imread(file))
    i_file+=1
train_array = np.asarray(train_array)

In [14]:
### Load dev images from directory
dev_array = []
i_file = 0
for f in range(N_dev_samples):
    file = dev_data_dir+'Im'+str(i_file)+'.png'
    dev_array.append(cv2.imread(file))
    i_file+=1
dev_array = np.asarray(dev_array)

In [19]:
### Load test images from directory
test_array = []
i_file = 0
for f in range(N_test_samples):
    file = test_data_dir+'Im'+str(i_file)+'.png'
    test_array.append(cv2.imread(file))
    i_file+=1
test_array = np.asarray(test_array)

In [16]:
## Parse labels file
file = open('/home/asya/Documents/CodePractice/SpeedChallenge/speed_challenge_2017/data/Labels_train.txt', 'r') 
labels_train = file.read().split()
file.close()
labels_train = np.asarray(labels_train).astype('float32')

## Parse labels file
file = open('/home/asya/Documents/CodePractice/SpeedChallenge/speed_challenge_2017/data/Labels_dev.txt', 'r') 
labels_dev = file.read().split()
file.close()
labels_dev = np.asarray(labels_dev).astype('float32')


file = open('/home/asya/Documents/CodePractice/SpeedChallenge/speed_challenge_2017/data/T_train.txt', 'r') 
T_train = file.read().split()
file.close()
T_train = np.asarray(T_train).astype('float32')

file = open('/home/asya/Documents/CodePractice/SpeedChallenge/speed_challenge_2017/data/T_dev.txt', 'r') 
T_dev = file.read().split()
file.close()
T_dev = np.asarray(T_dev).astype('float32')

In [17]:
def model(input_shape):
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)
    
    # Zero-Padding: pads the border of X_input with zeroes
   # X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv0')(X_input)
    X = BatchNormalization(axis = -1, name = 'bn0')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool')(X)

  #  # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv1')(X_input)
    X = BatchNormalization(axis = -1, name = 'bn1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool1')(X)

    X = Flatten()(X)

    ## NOTE: Option to ass a FC layer here but
    ## Model on the flow, angles and acc did better on a short training with no fully connected layer
    #X = Dense(5, name='d64')(X)
    #X = Activation('relu')(X)
    #sX = Dropout(0.5)(X)

    ## Output layer
    X = Dense(1, name='fc')(X)
    
    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='SpeedModel')
    
    return model


In [18]:
print(labels_train.shape)
print(np.asarray(train_array).shape)
#spModel = model(np.asarray(train_array[:,:,:,:][0]).shape)

(47922,)


NameError: name 'train_array' is not defined

In [ ]:
#spModel.compile(optimizer = 'adam', loss = "mean_squared_error", metrics = ['mse'])

In [ ]:
#history2step = spModel.fit(x = train_array[:,:,:,:], y = labels_train, epochs = 2, batch_size = 16)
#spModel.save_weights('N2.h5')

In [ ]:
#print(time.time()-tic)

In [ ]:
#preds = spModel.evaluate(x = dev_array[:,:,:,:], y = labels_dev)

In [ ]:
#print(preds[1])

In [ ]:
#pred_speed_t = spModel.predict(x = train_array[:,:,:,:])
#pred_speed_d = spModel.predict(x = dev_array[:,:,:,:])

In [ ]:
#plt.figure(figsize = (10,7))
#plt.plot(T_train,pred_speed_t,'b.')
#plt.plot(T_train,labels_train,'r.')

#plt.plot(T_dev,pred_speed_d,'g.')
#plt.plot(T_dev,labels_dev,'m.')

## NOTES ON MODELS

**3 input (angle,mag,acc), 2 conv layers:** 
mse 6.0719 after 2 epochs, 9.8 on dev set, predicted values are higher than expected maybe would do better with smoothing or a time dependence


**3 input (angle,mag,acc), 2 conv layers PLUS Dense64 layer:** Stopped - mse stays too high during training

**3 input (angle,mag,acc), 2 conv layers PLUS Dense5 layer:**
mse  after 2 epochs,  on dev set, 

## Getting creative: Try boosting the NN, by running 10 x and averaging the results

In [ ]:
pred_speed_T = []
pred_speed_D = []
tic = time.time()
for i in np.arange(10,15,1):
 #   idx = (np.random.random(10000)*N_train_samples).astype(int)
    spModel = model(np.asarray(train_array[0]).shape)
    spModel.compile(optimizer = 'adam', loss = "mean_squared_error", metrics = ['mse'])
    spModel.fit(x = train_array, y = labels_train, epochs = 2, batch_size = 16)
    spModel.save_weights('Boost_model_full'+str(i)+'.h5')
    
    print('Fit time: %.2f'%(time.time()-tic))
    tic = time.time()

Epoch 1/2
47922/47922 [==============================] - 5602s 117ms/step - loss: 97.3728 - mean_squared_error: 97.3728
Epoch 2/2
47922/47922 [==============================] - 8143s 170ms/step - loss: 20.7748 - mean_squared_error: 20.7748
Fit time: 13745.69
Epoch 1/2
47922/47922 [==============================] - 8032s 168ms/step - loss: 91.3669 - mean_squared_error: 91.3669
Epoch 2/2
47922/47922 [==============================] - 15299s 319ms/step - loss: 16.3041 - mean_squared_error: 16.3041
Fit time: 23332.70
Epoch 1/2
20448/47922 [===========>..................] - ETA: 41:04 - loss: 151.8375 - mean_squared_error: 151.8375

In [ ]:
pred_speed_T = []
for i in [0]: #np.arange(2,10,1):
  #  if i == 0: continue
    spModel = model(np.asarray(train_array[0]).shape)
    spModel.load_weights('Boost_model_full'+str(i)+'.h5')
    
    pred_speed_t = spModel.predict(x = train_array[:,:,:,:])
    
    with open('/home/asya/Documents/CodePractice/SpeedChallenge/speed_challenge_2017/data/predictions/train' + str(i) + '.txt', 'w') as f:
        for item in pred_speed_t:
            f.write("%s\n" % item[0])
    
#    pred_speed_T.append(pred_speed_t)
    
#pred_speed_T = np.asarray(pred_speed_T)[:,:,0]

In [20]:
for i in np.arange(0,10,1):
    spModel = model(np.asarray(test_array[0]).shape)
    spModel.load_weights('Boost_model_full'+str(i)+'.h5')
    pred_speed_d = spModel.predict(x = test_array[:,:,:,:])

    with open('/home/asya/Documents/CodePractice/SpeedChallenge/speed_challenge_2017/data/predictions/test' + str(i) + '.txt', 'w') as f:
        for item in pred_speed_d:
            f.write("%s\n" % item[0])

In [21]:
pred_speed_D = []
for i in np.arange(0,10,1):
    spModel = model(np.asarray(dev_array[0]).shape)
    spModel.load_weights('Boost_model_full'+str(i)+'.h5')
    
    pred_speed_d = spModel.predict(x = dev_array[:,:,:,:])

    with open('/home/asya/Documents/CodePractice/SpeedChallenge/speed_challenge_2017/data/predictions/dev' + str(i) + '.txt', 'w') as f:
        for item in pred_speed_d:
            f.write("%s\n" % item[0])
    
    pred_speed_D.append(pred_speed_d)
    
pred_speed_D = np.asarray(pred_speed_D)[:,:,0]

In [ ]:
print(array_boost_T.shape)

In [ ]:
array_boost_D = np.mean(pred_speed_D,axis = 0)
array_boost_T = np.mean(pred_speed_T,axis = 0)


from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from scipy.signal import medfilt
                     
for i in range(len(pred_speed_T)):
    mse_train = np.mean((pred_speed_T[i,:]-labels_train)**2)
    #mse_dev = np.mean((pred_speed_D[i,:]-labels_dev)**2)
    mse_dev = 0
    print('Train %.2f, Test %.2f'%(mse_train,mse_dev))
    
print(np.mean((array_boost_T-labels_train)**2))

print(np.mean((array_boost_D-labels_dev)**2))

#data = np.hstack((T_train, array_boost_T))
#model = SimpleExpSmoothing(data)
#model_fit = model.fit()
#yhat = model_fit.predict(T_train)
yhat = medfilt(array_boost_D,25)
print(np.mean((labels_dev-yhat)**2))

In [ ]:
plt.figure(figsize = (10,7))
plt.plot(T_train,array_boost_T,'b.')
plt.plot(T_train,yhat,'m.')
plt.plot(T_train,labels_train,'r.')

plt.plot(T_dev,array_boost_D,'g.')
plt.plot(T_dev,labels_dev,'r.')
plt.plot(T_dev,yhat,'b.')